In [ ]:
# make sure CUDA is installed
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
# make sure you have a GPU runtime (if this fails go to runtime -> change runtime type)
!nvidia-smi

Tue Apr 30 02:51:37 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# CUDA in Jupyter helpers
!pip install nvcc4jupyter
%load_ext nvcc4jupyter
# to learn about how to do more fancy things with CUDA using this API see:
# https://nvcc4jupyter.readthedocs.io/en/latest/index.html

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpry0yn6zu".


### PCR code for size n matrix



In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>
#include <stdlib.h>
#include <time.h>


__host__
void generate_tridiagonal_system(int n, float a[], float b[], float c[], float d[]) {
    // Seed the random number generator for variability in results
    // srand(time(NULL));

    for (int i = 0; i < n; i++) {
        // Generate random values for a, b, c, and d
        a[i] = (i > 0) ? rand() % 100 + 1 : 0;  // Upper diagonal (no entry at i=0)
        c[i] = (i < n-1) ? rand() % 100 + 1 : 0;  // Lower diagonal (no entry at i=n-1)
        b[i] = a[i] + c[i] + rand() % 100 + 50;  // Ensure diagonal dominance
        d[i] = rand() % 100 + 1;
    }
}

__global__ void list_print(int nmax, float * in) {
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    printf("Thread %i shows %f \n", i, in[i]);
}


__global__ void Solve_Kernel(
    float * alist, float * blist, float * clist, float * dlist, float * xlist,
    int iter_max, int DMax) {

    int idx_row = blockIdx.x*blockDim.x + threadIdx.x;
    int row_max = DMax - 1;

    int stride = 1;
    int next_stride = stride;

    float a1, b1, c1, d1;
    float k01, k21, c01, a21, d01, d21;

    bool next_or_ot = true;
    int accum;

    for (int iter = 0; iter < iter_max; iter++) {

        if ( next_or_ot ) {

            next_stride = stride<<1;

            // 1    for updating 'a'
            if ((idx_row - stride)<0) {
            // 1.1  if it is the 'first' line
                a1 = 0.0f;
                k01 = 0.0f;
                c01 = 0.0f;
                d01 = 0.0f;
            } else if ((idx_row - next_stride)<0) {
            // 1.2  if no place for 'a'
                a1 = 0.0f;
                k01 = alist[idx_row]/blist[idx_row - stride];
                c01 = clist[idx_row - stride]*k01;
                d01 = dlist[idx_row - stride]*k01;
            } else {
            // 1.3  for rest general rows
                k01 = alist[idx_row]/blist[idx_row - stride];
                a1 = -alist[idx_row - stride]*k01;
                c01 = clist[idx_row - stride]*k01;
                d01 = dlist[idx_row - stride]*k01;
            }

            // 2    for updating 'c'
            if ((idx_row + stride)>row_max) {
            // 2.1  if it is the 'last' line
                c1 = 0.0f;
                k21 = 0.0f;
                a21 = 0.0f;
                d21 = 0.0f;
            } else if ((idx_row + next_stride)>row_max) {
                c1 = 0.0f;
                k21 = clist[idx_row]/blist[idx_row + stride];
                a21 = alist[idx_row + stride]*k21;
                d21 = dlist[idx_row + stride]*k21;
            } else {
                k21 = clist[idx_row]/blist[idx_row + stride];
                c1 = -clist[idx_row + stride]*k21;
                a21 = alist[idx_row + stride]*k21;
                d21 = dlist[idx_row + stride]*k21;
            }
            // 3   for updating 'b'
            b1 = blist[idx_row] - c01 - a21;
            // 4   for updating 'd'
            d1 = dlist[idx_row] - d01 - d21;

            stride = next_stride;

            //Determine if this line has reached the bi-set
            int pos = idx_row-2*stride;
            accum = 0;
            for ( size_t iter = 0; iter<5; iter++ ) {
                if (pos >=0 && pos < DMax) accum++;
                pos+=stride;
            }
            if (accum < 3) {
                next_or_ot = false;//Turn of for ever
            }

        }

        __syncthreads();

        alist[idx_row] = a1;
        blist[idx_row] = b1;
        clist[idx_row] = c1;
        dlist[idx_row] = d1;

    }

    if ( accum==1 ) {
        xlist[idx_row] = dlist[idx_row] / blist[idx_row];
    } else if ( (idx_row-stride)<0 ) {
        int i = idx_row; int k = idx_row+stride;
        float f = clist[i]/blist[k];
        xlist[i] = (dlist[i]-dlist[k]*f)/(blist[i]-alist[k]*f);
    } else {
        int i = idx_row - stride; int k = idx_row;
        float f = alist[k]/blist[i];
        xlist[k] = (dlist[k]-dlist[i]*f)/(blist[k]-clist[i]*f);
    }

}

__host__
int main() {

    // Open a CSV file to save the execution times
    FILE *fp = fopen("execution_times.csv", "w");
    if (fp == NULL) {
        fprintf(stderr, "Failed to open file for writing.\n");
        return -1;
    }
    fprintf(fp, "System Size,Execution Time (ms)\n"); // Write the CSV header

    int sizes[] = {2, 5, 10, 100, 1000, 2000, 5000, 10000};
    int num_sizes = sizeof(sizes) / sizeof(sizes[0]);  // Number of different system sizes

    int num_systems = 1000;
    for (int idx = 0; idx < num_sizes; idx++) {
        int n = sizes[idx];  // System size for this iteration
        printf("System size: %d\n", n);
        srand(time(NULL));
        float totaltime = 0;
        for (int i = 0; i <= num_systems; i++) {
            float *h_a = (float *) malloc(n * sizeof(float));  // Upper diagonal
            float *h_b = (float *) malloc(n * sizeof(float));  // Main diagonal
            float *h_c = (float *) malloc(n * sizeof(float));  // Lower diagonal
            float *h_d = (float *) malloc(n * sizeof(float));  // Right-hand side vector
            float h_x[n];

            generate_tridiagonal_system(n, h_a, h_b, h_c, h_d);

            // Device arrays
            float *d_a, *d_b, *d_c, *d_d, *d_x;

            // Allocate memory on the device
            cudaMalloc(&d_a, n * sizeof(float));
            cudaMalloc(&d_b, n * sizeof(float));
            cudaMalloc(&d_c, n * sizeof(float));
            cudaMalloc(&d_d, n * sizeof(float));
            cudaMalloc(&d_x, n * sizeof(float));

            // Copy data from host to device
            cudaMemcpy(d_a, h_a, n * sizeof(float), cudaMemcpyHostToDevice);
            cudaMemcpy(d_b, h_b, n * sizeof(float), cudaMemcpyHostToDevice);
            cudaMemcpy(d_c, h_c, n * sizeof(float), cudaMemcpyHostToDevice);
            cudaMemcpy(d_d, h_d, n * sizeof(float), cudaMemcpyHostToDevice);
            cudaMemcpy(d_x, h_x, n * sizeof(float), cudaMemcpyHostToDevice);

            // Define kernel launch configuration
            int threadsPerBlock = 256;
            int blocks = (n + threadsPerBlock - 1) / threadsPerBlock;

            // Setting up timing
            cudaEvent_t start, stop;
            cudaEventCreate(&start);
            cudaEventCreate(&stop);

            // Launch the kernel
            cudaEventRecord(start);
            Solve_Kernel<<<blocks, threadsPerBlock>>>(d_a, d_b, d_c, d_d, d_x, 20, n);

            cudaEventRecord(stop);

            cudaEventSynchronize(stop);
            float milliseconds = 0;
            cudaEventElapsedTime(&milliseconds, start, stop);
            //printf("Execution time: %f milliseconds\n", milliseconds);
            //fprintf(fp, "Execution time for system %d: %f milliseconds\n", i, milliseconds);
            if(i != 0) {
                totaltime += milliseconds;
            }

            // Copy results back to host
            cudaMemcpy(h_x, d_x, n * sizeof(float), cudaMemcpyDeviceToHost);

            // Output the results
            /*
            printf("Solution vector x:\n");
            fprintf(fp, "Solution vector for system %d:\n", i);
            for (int i = 0; i < n; i++) {
                printf("%f, ", h_x[i]);
                fprintf(fp, "%f ", h_x[i]);
            }

            printf("\n\n");
            fprintf(fp, "\n\n");
            */

            // Free device memory
            cudaFree(d_a);
            cudaFree(d_b);
            cudaFree(d_c);
            cudaFree(d_d);
            cudaFree(d_x);

            free(h_a);
            free(h_b);
            free(h_c);
            free(h_d);

            // Destroy the events
            cudaEventDestroy(start);
            cudaEventDestroy(stop);
        }

        printf("Average runtime: %f\n", totaltime/num_systems);
        fprintf(fp,"%d,%f\n", n,totaltime/num_systems);
    }

    fclose(fp);
    return 0;
}

System size: 2
Average runtime: 0.015898
System size: 5
Average runtime: 0.016212
System size: 10
Average runtime: 0.013581
System size: 100
Average runtime: 0.012116
System size: 1000
Average runtime: 0.016177
System size: 2000
Average runtime: 0.017492
System size: 5000
Average runtime: 0.019572
System size: 10000
Average runtime: 0.026478



# JESSICA: Using to test accuracy

In [ ]:
%%cuda
#include <stdio.h>
#include <cuda_runtime.h>
#include <stdlib.h>
#include <time.h>


__host__
void generate_tridiagonal_system(int n, float a[], float b[], float c[], float d[]) {
    // Seed the random number generator for variability in results
    // srand(time(NULL));

    for (int i = 0; i < n; i++) {
        // Generate random values for a, b, c, and d
        a[i] = (i > 0) ? rand() % 100 + 1 : 0;  // Upper diagonal (no entry at i=0)
        c[i] = (i < n-1) ? rand() % 100 + 1 : 0;  // Lower diagonal (no entry at i=n-1)
        b[i] = a[i] + c[i] + rand() % 100 + 50;  // Ensure diagonal dominance
        d[i] = rand() % 100 + 1;
    }
}

__global__ void list_print(int nmax, float * in) {
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    printf("Thread %i shows %f \n", i, in[i]);
}


__global__ void Solve_Kernel(
    float * alist, float * blist, float * clist, float * dlist, float * xlist,
    int iter_max, int DMax) {

    int idx_row = blockIdx.x*blockDim.x + threadIdx.x;
    int row_max = DMax - 1;

    int stride = 1;
    int next_stride = stride;

    float a1, b1, c1, d1;
    float k01, k21, c01, a21, d01, d21;

    bool next_or_ot = true;
    int accum;

    for (int iter = 0; iter < iter_max; iter++) {

        if ( next_or_ot ) {

            next_stride = stride<<1;

            // 1    for updating 'a'
            if ((idx_row - stride)<0) {
            // 1.1  if it is the 'first' line
                a1 = 0.0f;
                k01 = 0.0f;
                c01 = 0.0f;
                d01 = 0.0f;
            } else if ((idx_row - next_stride)<0) {
            // 1.2  if no place for 'a'
                a1 = 0.0f;
                k01 = alist[idx_row]/blist[idx_row - stride];
                c01 = clist[idx_row - stride]*k01;
                d01 = dlist[idx_row - stride]*k01;
            } else {
            // 1.3  for rest general rows
                k01 = alist[idx_row]/blist[idx_row - stride];
                a1 = -alist[idx_row - stride]*k01;
                c01 = clist[idx_row - stride]*k01;
                d01 = dlist[idx_row - stride]*k01;
            }

            // 2    for updating 'c'
            if ((idx_row + stride)>row_max) {
            // 2.1  if it is the 'last' line
                c1 = 0.0f;
                k21 = 0.0f;
                a21 = 0.0f;
                d21 = 0.0f;
            } else if ((idx_row + next_stride)>row_max) {
                c1 = 0.0f;
                k21 = clist[idx_row]/blist[idx_row + stride];
                a21 = alist[idx_row + stride]*k21;
                d21 = dlist[idx_row + stride]*k21;
            } else {
                k21 = clist[idx_row]/blist[idx_row + stride];
                c1 = -clist[idx_row + stride]*k21;
                a21 = alist[idx_row + stride]*k21;
                d21 = dlist[idx_row + stride]*k21;
            }
            // 3   for updating 'b'
            b1 = blist[idx_row] - c01 - a21;
            // 4   for updating 'd'
            d1 = dlist[idx_row] - d01 - d21;

            stride = next_stride;

            //Determine if this line has reached the bi-set
            int pos = idx_row-2*stride;
            accum = 0;
            for ( size_t iter = 0; iter<5; iter++ ) {
                if (pos >=0 && pos < DMax) accum++;
                pos+=stride;
            }
            if (accum < 3) {
                next_or_ot = false;//Turn of for ever
            }

        }

        __syncthreads();

        alist[idx_row] = a1;
        blist[idx_row] = b1;
        clist[idx_row] = c1;
        dlist[idx_row] = d1;

    }

    if ( accum==1 ) {
        xlist[idx_row] = dlist[idx_row] / blist[idx_row];
    } else if ( (idx_row-stride)<0 ) {
        int i = idx_row; int k = idx_row+stride;
        float f = clist[i]/blist[k];
        xlist[i] = (dlist[i]-dlist[k]*f)/(blist[i]-alist[k]*f);
    } else {
        int i = idx_row - stride; int k = idx_row;
        float f = alist[k]/blist[i];
        xlist[k] = (dlist[k]-dlist[i]*f)/(blist[k]-clist[i]*f);
    }

}

__host__
int main() {

    // Open a CSV file to save the execution times
    FILE *fp = fopen("results.txt", "w");
    if (fp == NULL) {
        fprintf(stderr, "Failed to open file for writing.\n");
        return -1;
    }

    int sizes[] = {100};
    int num_sizes = sizeof(sizes) / sizeof(sizes[0]);  // Number of different system sizes

    int num_systems = 99;
    fprintf(fp, "%d\n", num_systems+1);
    for (int idx = 0; idx < num_sizes; idx++) {
        int n = sizes[idx];  // System size for this iteration
        srand(time(NULL));
        float totaltime = 0;
        for (int i = 0; i <= num_systems; i++) {
            float *h_a = (float *) malloc(n * sizeof(float));  // Upper diagonal
            float *h_b = (float *) malloc(n * sizeof(float));  // Main diagonal
            float *h_c = (float *) malloc(n * sizeof(float));  // Lower diagonal
            float *h_d = (float *) malloc(n * sizeof(float));  // Right-hand side vector
            float h_x[n];

            generate_tridiagonal_system(n, h_a, h_b, h_c, h_d);

            for (int i = 0; i < n; i++) {
                if(i == 0) {
                    printf("%.0f, ", h_a[i]);
                }
                else if(i == n-1) {
                    printf("%.0f ", h_a[i]);
                    fprintf(fp, "%.0f 0", h_a[i]);
                } else {
                    printf("%.0f, ", h_a[i]);
                    fprintf(fp, "%.0f ", h_a[i]);
                }
            }
            printf("\n");
            fprintf(fp, "\n");
            for (int i = 0; i < n; i++) {
                if(i == n-1) {
                    printf("%.0f ", h_b[i]);
                    fprintf(fp, "%.0f ", h_b[i]);
                } else {
                    printf("%.0f, ", h_b[i]);
                    fprintf(fp, "%.0f ", h_b[i]);
                }
            }
            printf("\n");
            fprintf(fp, "\n");
            for (int i = 0; i < n; i++) {
                if(i == n-1) {
                    printf("%.0f ", h_c[i]);
                    fprintf(fp, "%.0f ", h_c[i]);
                } else {
                    printf("%.0f, ", h_c[i]);
                    fprintf(fp, "%.0f ", h_c[i]);
                }
            }
            printf("\n");
            fprintf(fp, "\n");
            for (int i = 0; i < n; i++) {
                if(i == n-1) {
                    printf("%.0f ", h_d[i]);
                    fprintf(fp, "%.0f ", h_d[i]);
                } else {
                    printf("%.0f, ", h_d[i]);
                    fprintf(fp, "%.0f ", h_d[i]);
                }
            }
            printf("\n");
            fprintf(fp, "\n");
            // Device arrays
            float *d_a, *d_b, *d_c, *d_d, *d_x;

            // Allocate memory on the device
            cudaMalloc(&d_a, n * sizeof(float));
            cudaMalloc(&d_b, n * sizeof(float));
            cudaMalloc(&d_c, n * sizeof(float));
            cudaMalloc(&d_d, n * sizeof(float));
            cudaMalloc(&d_x, n * sizeof(float));

            // Copy data from host to device
            cudaMemcpy(d_a, h_a, n * sizeof(float), cudaMemcpyHostToDevice);
            cudaMemcpy(d_b, h_b, n * sizeof(float), cudaMemcpyHostToDevice);
            cudaMemcpy(d_c, h_c, n * sizeof(float), cudaMemcpyHostToDevice);
            cudaMemcpy(d_d, h_d, n * sizeof(float), cudaMemcpyHostToDevice);
            cudaMemcpy(d_x, h_x, n * sizeof(float), cudaMemcpyHostToDevice);

            // Define kernel launch configuration
            int threadsPerBlock = 256;
            int blocks = (n + threadsPerBlock - 1) / threadsPerBlock;

            // Setting up timing
            cudaEvent_t start, stop;
            cudaEventCreate(&start);
            cudaEventCreate(&stop);

            // Launch the kernel
            cudaEventRecord(start);
            Solve_Kernel<<<blocks, threadsPerBlock>>>(d_a, d_b, d_c, d_d, d_x, 20, n);
            cudaEventRecord(stop);
            cudaEventSynchronize(stop);
            float milliseconds = 0;
            cudaEventElapsedTime(&milliseconds, start, stop);


            // Copy results back to host
            cudaMemcpy(h_x, d_x, n * sizeof(float), cudaMemcpyDeviceToHost);

            // Output the results

            printf("Solution vector x:\n");
            for (int i = 0; i < n; i++) {
                printf("%f, ", h_x[i]);
                fprintf(fp, "%f ", h_x[i]);
            }


            printf("\n\n");
            fprintf(fp, "\n");

            // Free device memory
            cudaFree(d_a);
            cudaFree(d_b);
            cudaFree(d_c);
            cudaFree(d_d);
            cudaFree(d_x);

            free(h_a);
            free(h_b);
            free(h_c);
            free(h_d);

            // Destroy the events
            cudaEventDestroy(start);
            cudaEventDestroy(stop);
        }
    }

    fclose(fp);
    return 0;
}

0, 53, 35, 21, 20, 2, 99, 51, 38, 50, 94, 13, 67, 42, 11, 59, 47, 69, 49, 3, 3, 10, 39, 47, 24, 62, 42, 22, 20, 75, 14, 20, 37, 68, 83, 2, 82, 61, 85, 33, 40, 90, 8, 97, 57, 33, 86, 72, 72, 36, 13, 18, 69, 85, 61, 78, 89, 51, 57, 58, 44, 24, 52, 24, 28, 36, 31, 39, 7, 2, 7, 7, 55, 42, 83, 69, 30, 82, 19, 40, 5, 37, 76, 65, 61, 8, 41, 28, 36, 86, 87, 29, 13, 92, 78, 9, 50, 76, 46, 94 
123, 244, 177, 160, 212, 181, 249, 267, 175, 218, 300, 203, 187, 171, 148, 255, 173, 227, 145, 134, 94, 113, 176, 226, 147, 190, 200, 108, 149, 183, 107, 127, 222, 222, 219, 203, 182, 230, 250, 128, 249, 233, 109, 284, 227, 179, 237, 240, 248, 258, 179, 193, 188, 242, 249, 252, 262, 214, 205, 223, 165, 129, 214, 168, 157, 189, 167, 121, 119, 136, 180, 168, 154, 239, 199, 254, 156, 297, 216, 169, 218, 230, 198, 231, 170, 159, 200, 164, 180, 182, 280, 258, 122, 207, 188, 136, 193, 240, 171, 203 
52, 98, 64, 45, 84, 75, 2, 87, 53, 79, 69, 84, 67, 58, 73, 94, 39, 29, 19, 36, 1, 15, 54, 86, 33, 72, 86, 20, 9, 9

_______

In [ ]:



%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

__global__ void list_print(int nmax, float * in) {
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    printf("Thread %i shows %f \n", i, in[i]);
}


__global__ void Solve_Kernel(
    float * alist, float * blist, float * clist, float * dlist, float * xlist,
    int iter_max, int DMax) {

    int idx_row = blockIdx.x*blockDim.x + threadIdx.x;
    int row_max = DMax - 1;

    int stride = 1;
    int next_stride = stride;

    float a1, b1, c1, d1;
    float k01, k21, c01, a21, d01, d21;

    bool next_or_ot = true;
    int accum;

    for (int iter = 0; iter < iter_max; iter++) {

        if ( next_or_ot ) {

            next_stride = stride<<1;

            // 1    for updating 'a'
            if ((idx_row - stride)<0) {
            // 1.1  if it is the 'first' line
                a1 = 0.0f;
                k01 = 0.0f;
                c01 = 0.0f;
                d01 = 0.0f;
            } else if ((idx_row - next_stride)<0) {
            // 1.2  if no place for 'a'
                a1 = 0.0f;
                k01 = alist[idx_row]/blist[idx_row - stride];
                c01 = clist[idx_row - stride]*k01;
                d01 = dlist[idx_row - stride]*k01;
            } else {
            // 1.3  for rest general rows
                k01 = alist[idx_row]/blist[idx_row - stride];
                a1 = -alist[idx_row - stride]*k01;
                c01 = clist[idx_row - stride]*k01;
                d01 = dlist[idx_row - stride]*k01;
            }

            // 2    for updating 'c'
            if ((idx_row + stride)>row_max) {
            // 2.1  if it is the 'last' line
                c1 = 0.0f;
                k21 = 0.0f;
                a21 = 0.0f;
                d21 = 0.0f;
            } else if ((idx_row + next_stride)>row_max) {
                c1 = 0.0f;
                k21 = clist[idx_row]/blist[idx_row + stride];
                a21 = alist[idx_row + stride]*k21;
                d21 = dlist[idx_row + stride]*k21;
            } else {
                k21 = clist[idx_row]/blist[idx_row + stride];
                c1 = -clist[idx_row + stride]*k21;
                a21 = alist[idx_row + stride]*k21;
                d21 = dlist[idx_row + stride]*k21;
            }
            // 3   for updating 'b'
            b1 = blist[idx_row] - c01 - a21;
            // 4   for updating 'd'
            d1 = dlist[idx_row] - d01 - d21;

            stride = next_stride;

            //Determine if this line has reached the bi-set
            int pos = idx_row-2*stride;
            accum = 0;
            for ( size_t iter = 0; iter<5; iter++ ) {
                if (pos >=0 && pos < DMax) accum++;
                pos+=stride;
            }
            if (accum < 3) {
                next_or_ot = false;//Turn of for ever
            }

        }

        __syncthreads();

        alist[idx_row] = a1;
        blist[idx_row] = b1;
        clist[idx_row] = c1;
        dlist[idx_row] = d1;

    }

    if ( accum==1 ) {
        xlist[idx_row] = dlist[idx_row] / blist[idx_row];
    } else if ( (idx_row-stride)<0 ) {
        int i = idx_row; int k = idx_row+stride;
        float f = clist[i]/blist[k];
        xlist[i] = (dlist[i]-dlist[k]*f)/(blist[i]-alist[k]*f);
    } else {
        int i = idx_row - stride; int k = idx_row;
        float f = alist[k]/blist[i];
        xlist[k] = (dlist[k]-dlist[i]*f)/(blist[k]-clist[i]*f);
    }

}

__host__
int main() {
    int diagonal_size = 4;

    // Host arrays
    float h_a[] = {0, -1, -1, -1};
    float h_b[] = {2, 2, 2, 1};
    float h_c[] = {-1, -1, -1, 0};
    float h_d[] = {0, 0, 1, 0};
    float h_x[diagonal_size];

    // Device arrays
    float *d_a, *d_b, *d_c, *d_d, *d_x;

    // Allocate memory on the device
    cudaMalloc(&d_a, diagonal_size * sizeof(float));
    cudaMalloc(&d_b, diagonal_size * sizeof(float));
    cudaMalloc(&d_c, diagonal_size * sizeof(float));
    cudaMalloc(&d_d, diagonal_size * sizeof(float));
    cudaMalloc(&d_x, diagonal_size * sizeof(float));

    // Copy data from host to device
    cudaMemcpy(d_a, h_a, diagonal_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, diagonal_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_c, h_c, diagonal_size * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_d, h_d, diagonal_size * sizeof(float), cudaMemcpyHostToDevice);

    // Define kernel launch configuration
    int threadsPerBlock = 256;
    int blocks = (diagonal_size + threadsPerBlock - 1) / threadsPerBlock;

    // Launch the kernel
    Solve_Kernel<<<blocks, threadsPerBlock>>>(d_a, d_b, d_c, d_d, d_x, 10, diagonal_size);

    // Copy results back to host
    cudaMemcpy(h_x, d_x, diagonal_size * sizeof(float), cudaMemcpyDeviceToHost);

    // Output the results
    printf("Solution vector x:\n");
    for (int i = 0; i < diagonal_size; i++) {
        printf("%f ", h_x[i]);
    }

    printf("\n");

    // Free device memory
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    cudaFree(d_d);
    cudaFree(d_x);

    return 0;
}

Solution vector x:
1.000000 2.000000 3.000000 3.000000 



____

In [ ]:
#include <iostream>
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <cstdlib>

#include "PCR_Class.h"

int main( ) {

    size_t diagonal_size = 9;

    PCR_Solver crs = PCR_Solver(diagonal_size);

    //Generate sampel data
    srand (time(NULL));

    thrust::device_vector<float> alist(diagonal_size);
    thrust::device_vector<float> blist(diagonal_size);
    thrust::device_vector<float> clist(diagonal_size);
    thrust::device_vector<float> dlist(diagonal_size);
    thrust::device_vector<float> xlist(diagonal_size);

    float * ptr_alist = thrust::raw_pointer_cast(alist.data());
    float * ptr_blist = thrust::raw_pointer_cast(blist.data());
    float * ptr_clist = thrust::raw_pointer_cast(clist.data());
    float * ptr_dlist = thrust::raw_pointer_cast(dlist.data());
    float * ptr_xlist = thrust::raw_pointer_cast(xlist.data());

    for (int i=0; i < diagonal_size; i++) {
        alist[i] = i+2;
        blist[i] = i+1;
        clist[i] = i+3;
        dlist[i] = i+10;//rand() % 100 + 1;
        xlist[i] = 0.0f;
    }

    alist[0] = float(0.0);
    clist[diagonal_size-1] = float(0.0);

    crs.Solve(ptr_alist, ptr_blist, ptr_clist, ptr_dlist, ptr_xlist);

    for (auto item : xlist) {
        std::cout << item << std::endl;
    }

    //for (size_t it=0; it<diagonal_size; it++) {
    //    std::cout << alist[it] << " " << blist[it] << " " << clist[it] << " " << xlist[it] << " " <<  dlist[it] << std::endl;
    //}

    return 0;

}

SyntaxError: invalid decimal literal (<ipython-input-18-2f4742f500ce>, line 34)

In [ ]:
%%cuda
#include <math.h>
#include <iostream>
#include <cuda_runtime.h>

// Define the data type T
typedef double T;

__global__
void pcrKernel(T* a, T* b, T* c, T* d, T* x, int sizeSystem, int delta) {
    int numSteps = 1;
    for (int j = 0; j < numSteps; j++) {
        int i = threadIdx.x;

        int iRight = i + delta;
        if (iRight >= sizeSystem) {
          iRight = sizeSystem - 1;
        }

        int iLeft = i - delta;
        if (iLeft < 0) {
            iLeft = 0;
        }

        T tmp1 = a[i] / b[iLeft];
        T tmp2 = c[i] / b[iRight];

        T bNew  = b[i] - c[iLeft] * tmp1 + a[iRight] * tmp2;
        T dNew  = d[i] - d[iLeft] * tmp1 - d[iRight] * tmp2;
        T aNew  = -a[iLeft] * tmp1;
        T cNew  = -c[iRight] * tmp2;

        __syncthreads();

        b[i] = bNew;
        d[i] = dNew;
        a[i] = aNew;
        c[i] = cNew;

        delta *= 2;

        __syncthreads();
    }

    if (threadIdx.x < delta) {
        int addr1 = threadIdx.x;
        int addr2 = threadIdx.x + delta;
        T tmp3 = b[addr2] * b[addr1] - c[addr1] * a[addr2];
        x[addr1] = (b[addr2] * d[addr1] - c[addr1] * d[addr2]) / tmp3;
        x[addr2] = (d[addr2] * b[addr1] - d[addr1] * a[addr2]) / tmp3;
    }
}


__host__
int main() {
    T a[] = {0, -1, -1, -1};
    T b[] = {2, 2, 2, 1};
    T c[] = {-1, -1, -1, 0};
    T d[] = {0, 0, 1, 0};
    T x[4];

    int numSystems = 1;
    int sizeSystem = 4;
    int delta = 1;

    dim3 grid(numSystems, 1, 1);
    dim3 threads(sizeSystem, 1, 1);
    int sizeSharedMemory = sizeSystem * 5 * sizeof(T);

    T *d_a, *d_b, *d_c, *d_d, *d_x;
    cudaMalloc((void**)&d_a, sizeSystem * sizeof(T));
    cudaMalloc((void**)&d_b, sizeSystem * sizeof(T));
    cudaMalloc((void**)&d_c, sizeSystem * sizeof(T));
    cudaMalloc((void**)&d_d, sizeSystem * sizeof(T));
    cudaMalloc((void**)&d_x, sizeSystem * sizeof(T));
    cudaMemcpy(d_a, a, sizeSystem * sizeof(T), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, sizeSystem * sizeof(T), cudaMemcpyHostToDevice);
    cudaMemcpy(d_c, c, sizeSystem * sizeof(T), cudaMemcpyHostToDevice);
    cudaMemcpy(d_d, d, sizeSystem * sizeof(T), cudaMemcpyHostToDevice);

    pcrKernel<<<grid, threads, sizeSharedMemory>>>(d_a, d_b, d_c, d_d, d_x, sizeSystem, delta);
    cudaError_t cuda_error = cudaGetLastError();
    if(cuda_error != cudaSuccess) {
        // Kernel launch failed, print error message
        std::cerr << "CUDA kernel launch failed: " << cudaGetErrorString(cuda_error) << std::endl;
        exit(EXIT_FAILURE);
    }

    cudaDeviceSynchronize();
    cudaMemcpy(x, d_x, sizeSystem * sizeof(T), cudaMemcpyDeviceToHost);

    for (int i = 0; i < sizeSystem; ++i) {
        std::cout << x[i] << " ";
    }
    std::cout << std::endl;

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
    cudaFree(d_d);
    cudaFree(d_x);

    return 0;
}

0.0833333 0.666667 0.416667 1.66667 



In [ ]:
%%cuda
#include <iostream>
#include <vector>
#include <cuda_runtime.h>

__global__ void cyclic_reduction_kernel(float* a, float* b, float* c, float* d, int delta, int size_system) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < size_system) {
        int i_right = i + delta;
        int i_left = i - delta;

        if (i_right >= size_system) i_right = size_system - 1;
        if (i_left < 0) i_left = 0;

        float tmp1 = a[i] / b[i_left];
        float tmp2 = c[i] / b[i_right];

        float b_new = b[i] - c[i_left] * tmp1 - a[i_right] * tmp2;
        float d_new = d[i] - d[i_left] * tmp1 - d[i_right] * tmp2;
        float a_new = -a[i_left] * tmp1;
        float c_new = -c[i_right] * tmp2;

        __syncthreads();

        b[i] = b_new;
        d[i] = d_new;
        a[i] = a_new;
        c[i] = c_new;

        __syncthreads();
    }
}

__global__ void solve_system_kernel(float* x, float* b, float* c, float* a, float* d, int delta, int size_system) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < delta) {
        int addr1 = i;
        int addr2 = i + delta;
        float tmp3 = b[addr2] * b[addr1] - c[addr1] * a[addr2];
        x[addr1] = (b[addr2] * d[addr1] - c[addr1] * d[addr2]) / tmp3;
        x[addr2] = (d[addr2] * b[addr1] - d[addr1] * a[addr2]) / tmp3;
    }
}

std::vector<float> parallel_cyclic_reduction_cuda(std::vector<float>& a, std::vector<float>& b, std::vector<float>& c, std::vector<float>& d, int num_steps) {
    int size_system = b.size();
    int delta = 1;

    float* a_device, * b_device, * c_device, * d_device;
    cudaMalloc(&a_device, size_system * sizeof(float));
    cudaMalloc(&b_device, size_system * sizeof(float));
    cudaMalloc(&c_device, size_system * sizeof(float));
    cudaMalloc(&d_device, size_system * sizeof(float));

    cudaMemcpy(a_device, a.data(), size_system * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(b_device, b.data(), size_system * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(c_device, c.data(), size_system * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_device, d.data(), size_system * sizeof(float), cudaMemcpyHostToDevice);

    for (int j = 0; j < num_steps; ++j) {
        cyclic_reduction_kernel<<<1, 4>>>(a_device, b_device, c_device, d_device, delta, size_system);
        delta *= 2;
        if (size_system <= delta) break;
    }

    std::vector<float> x(size_system);
    float* x_device;
    cudaMalloc(&x_device, size_system * sizeof(float));

    solve_system_kernel<<<1, 4>>>(x_device, b_device, c_device, a_device, d_device, delta, size_system);

    cudaMemcpy(x.data(), x_device, size_system * sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(a_device);
    cudaFree(b_device);
    cudaFree(c_device);
    cudaFree(d_device);
    cudaFree(x_device);

    return x;
}


int main() {
    std::vector<float> a = {0, -1, -1, -1};
    std::vector<float> b = {2, 2, 2, 1};
    std::vector<float> c = {-1, -1, -1, 0};
    std::vector<float> d = {0, 0, 1, 0};
    int num_steps = 1;

    std::vector<float> result = parallel_cyclic_reduction_cuda(a, b, c, d, num_steps);

    // Print the result
    std::cout << "Result:" << std::endl;
    for (int i = 0; i < result.size(); ++i) {
        std::cout << "x[" << i << "] = " << result[i] << std::endl;
    }


    return 0;
}

Result:
x[0] = 1
x[1] = 2
x[2] = 3
x[3] = 3



In [ ]:
%%cuda
#include <iostream>
#include <vector>
#include <cuda_runtime.h>

std::vector<float> parallel_cyclic_reduction_cuda(std::vector<float>& a, std::vector<float>& b, std::vector<float>& c, std::vector<float>& d, int system_size) {
    int num_steps = 1; // log2(system_size/2)
    int delta = 1;

    std::vector<float> x(system_size);

    float* d_a, d_b, d_c, d_d, d_x;
    cudaMalloc(&d_a, system_size * sizeof(float));
    cudaMalloc(&d_b, system_size * sizeof(float));
    cudaMalloc(&d_c, system_size * sizeof(float));
    cudaMalloc(&d_d, system_size * sizeof(float));
    cudaMalloc(&d_x, system_size * sizeof(float));


}

int main() {
    std::vector<float> a = {0, -1, -1, -1};
    std::vector<float> b = {2, 2, 2, 1};
    std::vector<float> c = {-1, -1, -1, 0};
    std::vector<float> d = {0, 0, 1, 0};
    int system_size = b.size()

    std::vector<float> result = parallel_cyclic_reduction_cuda(a, b, c, d, system_size);

    std::cout << "Result:" << std::endl;
    for (int i = 0; i < result.size(); ++i) {
        std::cout << "x[" << i << "] = " << result[i] << std::endl;
    }


    return 0;
}